In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm, trange
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
import librosa
import torch
import laion_clap
# from extraction.vgg_sound import *
# 在aws notebook环境需要加上这个
import sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "..")))

from audioldm import image_to_audio, build_model, clap_to_audio
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import soundfile as sf 





/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:

from models import *
from vgg_dataset import *
from tqdm import trange
from infer import *

Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
vgg_video_path = "/mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video/"
gen_output_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/gen_outputs/"
output_ori_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/ground_truth/"
zip_output_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/output_audio.zip"



In [5]:
# model = V2AMapperMLPImproved(input_dim=512, hidden_dim=512, output_dim=512).to(device)
model = V2AMapperMLPImproved(input_dim=512, hidden_dim=2048, output_dim=512).to(device)
model = V2AMapperMLPImprovednoresidule(input_dim=512, hidden_dim=2048, output_dim=512).to(device)
# model = V2AMapperBiLSTM(input_dim=512, hidden_dim=1024, output_dim=512, lstm_hidden=256, lstm_layers=1,dropout=0.3).to(device)

In [6]:
# ckpt_path = "/home/ubuntu/best_model.pth"

ckpt_path = "/home/ubuntu/project/v2a-mapper/train/ckpts-lstm/best_model.pth"
ckpt_path = "/home/ubuntu/project/v2a-mapper/train/ckpts_mlp_res/best_model_mlp_res.pth"
ckpt_path = "/home/ubuntu/project/v2a-mapper/train/ckpts_mlp_wr/best_model_mlp_wores.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# model = V2AMapperMLP()
model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
model = model.to(device)

In [8]:

class LargeVideoTestDataset(Dataset):
    def __init__(self, data_dir, subset_ratio = 0.2, transform=None):
        """
        root_dir: 保存所有 .pth 文件的目录，每个文件对应一个 sample。
        transform: 如果需要对数据做预处理，可在这里传入。
        """
        super().__init__()
        # 仅收集当前目录下所有的 pth 文件列表
        file_list = []

        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".pth"):
                    file_list.append(os.path.join(root, file))

        # 仅使用前 20% 的数据
        num_samples = int(len(file_list) * subset_ratio)

        self.file_paths = sorted(file_list)[:num_samples]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 在这里按需读取，而不是一次性加载全部
        pth_path = self.file_paths[idx]
        sample_data = torch.load(pth_path)  
        clip_feat = sample_data['clip_features']  # (64, 512)
        clap_feat = sample_data['clap_features']  # (1, 512)
        id = sample_data['id']
        caption = sample_data['caption']

        if self.transform:
            clip_feat, clap_feat = self.transform((clip_feat, clap_feat))

        return clip_feat, clap_feat, id, caption

In [9]:
root = "/mnt/new_volume2/vgg_sound_emb"
partition = "test"
data_dir = f"{root}/{partition}"
# data_dir ="/mnt/new_volume2/ai_video_emb/"
vgg_sound = LargeVideoTestDataset(data_dir, subset_ratio = 1)

In [32]:
generate_audio(model,vgg_sound, num_gen = 100,
                    video_path = vgg_video_path,
                    output_path = gen_output_path,
                    output_ori_path = output_ori_path)

set()


DDIM Sampler: 100%|██████████| 200/200 [00:27<00:00,  7.25it/s]

chunk:   0%|          | 0/222 [00:00<?, ?it/s, now=None]
                                                        

  0%|          | 1/15209 [00:30<130:51:53, 30.98s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf58.20.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1729, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 09/15/2018.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 125, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 09/15/2018.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 10.05, 'bitrate': 2047, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile'

DDIM Sampler: 100%|██████████| 200/200 [00:27<00:00,  7.25it/s]

chunk:   0%|          | 0/199 [00:00<?, ?it/s, now=None]
                                                        

  0%|          | 2/15209 [01:01<130:25:37, 30.88s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1081, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 74, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 9.0, 'bitrate': 1165, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': 1081, 'video_fps': 30.0, 'default_audio_input_number': 0, 'def

...

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf58.20.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 671, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 96, 'metadata': {'Metadata': '', 'handler_name': 'IsoMedia File Produced by Google, 5-11-2011', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 10.03, 'bitrate': 848, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Constrained Baseline)', 'video_size': [640, 360], 'vide

DDIM Sampler: 100%|██████████| 200/200 [00:27<00:00,  7.24it/s]

chunk:   0%|          | 0/221 [00:00<?, ?it/s, now=None]
                                                        

  1%|          | 99/15209 [50:52<129:18:09, 30.81s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf58.20.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 352], 'bitrate': 615, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 12/08/2019.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 95, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 12/08/2019.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 10.02, 'bitrate': 834, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '

DDIM Sampler: 100%|██████████| 200/200 [00:27<00:00,  7.23it/s]

chunk:   0%|          | 0/209 [00:00<?, ?it/s, now=None]
                                                        

  1%|          | 99/15209 [51:23<130:42:58, 31.14s/it]

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 360], 'bitrate': 795, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 76, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 9.47, 'bitrate': 878, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [480, 360], 'video_bitrate': 795, 'video_fps': 30.0, 'default_audio_input_number': 0, 'default

In [33]:
# dummy_generate_audio(model,vgg_sound, num_gen = 100,
#                     video_path = vgg_video_path,
#                     output_path = '/',
#                     output_ori_path = '/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/ori_videos/')

In [2]:
gen_output_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/gen_outputs/"
output_ori_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/ground_truth/"
zip_output_path = "/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/output_audio.zip"


In [3]:
import zipfile
import os

def zip_folders_with_root(folders, zip_filename):
    """
    Compress multiple folders into a single zip file, preserving top-level folder names.
    
    :param folders: List of folder paths to include.
    :param zip_filename: The output zip file path.
    """
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder in folders:
            folder_base  = os.path.basename(folder.rstrip("/"))
            for root, dirs, files in os.walk(folder):
                for file in files:
                    full_path = os.path.join(root, file)
                    rel_path = os.path.relpath(full_path, start=os.path.dirname(folder))
                    zipf.write(full_path, arcname=rel_path)

zip_folders_with_root([gen_output_path[:-1], output_ori_path[:-1]], zip_output_path)

print(f"✅ 成功压缩两个文件夹为 {zip_output_path}")


✅ 成功压缩两个文件夹为 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/output_audio.zip


In [12]:
# cnt = 0
# for clip_feat, clap_feat, id, caption in tqdm(test_loader):
#     if cnt <= 21:
#         cnt += 1
#         continue
#     # clip_feat: (1, 64, 512)
#     # clap_feat: (1, 512)
#     # id: (1,)
#     # caption: (1,)
#     print(clip_feat.shape, clap_feat.shape, id, caption)
#     clip_feat = clip_feat.to(device)  # (64, 512)
#     clap_feat = clap_feat.squeeze(0).to(device)  # (512)

#     gen_clap_feat = model(clip_feat).to(device=device)  # (1, 512)
#     # calculate cosine similarity
#     cos = nn.CosineSimilarity(dim=0)  # compare whole vectors
#     cos_sim = cos(gen_clap_feat.squeeze(0), clap_feat.squeeze(0).squeeze(0))  # both are (512,)

#     print("cosine similarity:", cos_sim)
#     # print(cos_sim.shape)
#     # print(cos_sim)
#     image2audio(
#         file_name="/home/ubuntu/project/v2a-mapper/train/test_inf/test_ag/test_outputs/" + id[0][:-4] + "_gen.wav",
#         images=gen_clap_feat.to(device=device),
#         duration=10,
#         guidance_scale=2.5,
#         random_seed=42,
#         n_candidates=3
#     )
#     # !ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0][:-4]}.wav

#     cnt += 1
#     if cnt == 70:
#         break

In [13]:
# cnt = 0
# for clip_feat, clap_feat, id, caption in tqdm(test_loader):
#     # clip_feat: (1, 64, 512)
#     # clap_feat: (1, 512)
#     # id: (1,)
#     # caption: (1,)
#     print(clip_feat.shape, clap_feat.shape, id, caption)
#     clip_feat = clip_feat.to(device)  # (64, 512)
#     clap_feat = clap_feat.squeeze(0).to(device)  # (512)

#     gen_clap_feat = model(clip_feat).to(device=device)  # (1, 512)
#     # calculate cosine similarity
#     cos = nn.CosineSimilarity(dim=0)  # compare whole vectors
#     cos_sim = cos(gen_clap_feat.squeeze(0), clap_feat.squeeze(0).squeeze(0))  # both are (512,)

#     print("cosine similarity:", cos_sim)
#     # print(cos_sim.shape)
#     # print(cos_sim)
#     image2audio(
#         file_name="/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/" + id[0][:-4] + "_gen.wav",
#         images=gen_clap_feat.to(device=device),
#         duration=10,
#         guidance_scale=2.5,
#         random_seed=42,
#         n_candidates=3
#     )
#     !ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0][:-4]}.wav

#     cnt += 1
#     if cnt == 20:
#         break